# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [5]:
data = pd.read_csv('previsao_de_renda.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

# 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

In [10]:
import numpy as np
import statsmodels.api as sm
import patsy

In [18]:
# Crie a matriz de design
y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data=data, return_type='dataframe')

In [19]:
# Ajuste o modelo
model = sm.OLS(y, X)
result = model.fit()

In [20]:
# Visualize os resultados
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Fri, 16 Feb 2024   Prob (F-statistic):               0.00
Time:                        10:37:30   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

Faz sentido, sim. Posse de veiculo, imovel, idade e tempo de emprego eh estadisticamente significativo

# 2. Remova a variável menos significante e analise:

In [32]:
# Encontre o p-valor mais alto
max_p_value = result.pvalues.idxmax()

In [33]:
# Remova a variável correspondente
X = X.drop(columns=[max_p_value])

In [34]:
# Ajuste o modelo novamente
model = sm.OLS(y, X)
result = model.fit()

In [35]:
# Visualize os resultados atualizados
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     313.7
Date:                Fri, 16 Feb 2024   Prob (F-statistic):               0.00
Time:                        10:44:37   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12404   BIC:                         2.735e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [ ]:
Nao mudou quase nada. Nenhum parametro mudou muito. 

# 3. Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

In [41]:
# Limiar de significância
significance_threshold = 0.05

while True:
    # Ajustar o modelo
    model = sm.OLS(y, X)
    result = model.fit()
    
    # Encontrar a primeira variável com p-valor maior que o limiar de significância
    max_p_value = result.pvalues.max()
    
    # Se todos os p-valores forem menores que o limiar de significância, saia do loop
    if max_p_value < significance_threshold:
        break
        
    # Encontrar a variável correspondente ao primeiro p-valor maior que o limiar de significância
    variable_to_remove = result.pvalues[result.pvalues == max_p_value].index[0]
    
    # Remover a variável menos significativa
    X = X.drop(columns=[variable_to_remove])
    
# Visualize os resultados atualizados
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     530.1
Date:                Fri, 16 Feb 2024   Prob (F-statistic):               0.00
Time:                        10:50:58   Log-Likelihood:                -13572.
No. Observations:               12427   AIC:                         2.717e+04
Df Residuals:                   12413   BIC:                         2.728e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

O R-square e Adj contunia igual, porem agora visualmente eh mas limpo e facil pra escolher as variaveis com maior significancia